# An example of neutrino propagation simulation through flexOPT.jl 
Some of functions are the fruit of works from Estelle Salomé (July-August 2025) and TIPE (2025-2026).

In [1]:

using Pkg

cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv" # 1D Earth/Mars/Moon models are defined

include("../src/commonBatchs.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

include("../src/Neurthino.jl")

using .commonBatchs, .planet1D, .GeoPoints
using Colors



  Activating 

  1.742301 seconds (3.84 M allocations: 173.684 MiB, 1.56% gc time, 99.98% compilation time)


project at `~/Documents/Github/flexOPT`


## making 2D(/3D) box with the aid of GeoPoint.jl (not necessary for geodynamic models)

Please refer to getRegionalBox.ipynb if you have some questions 

In [2]:
set_default_planet!(:SphericalEarth) # SphericalEarth means a perfect sphere with a radius of 6371000.0 m

Ellipsoid(SphericalEarth)

In [3]:
# two (extreme) points that can define the slice (or the x-y local plane for 3D box)
p1 = GeoPoint(0.0,0.0,30.e3) # lat, lon in degrees, altitude in metre
p2 = GeoPoint(0.0,180.0,30.e3) # 


Δx = 25.e3 # in metre
Δz = 25.e3

altMax = 30.e3 # in metre
altMin = -(6371.e3*2)-30.e3 # in metre

-1.2772e7

In [4]:
boxGrids=constructLocalBox(p1,p2,Δx,Δz,altMin,altMax)

(allGridsInGeoPoints = GeoPoint[GeoPoint(-89.82730758109501, -90.0, 30000.00063271541, [0.0, -19292.913182086468, -6.400970925695646e6], 6.401000000632715e6) GeoPoint(-89.8273044223904, -90.0, 5000.000635194592, [0.0, -19217.91351958419, -6.375971038194885e6], 6.376000000635195e6) … GeoPoint(89.82892218561636, 90.0, 3000.000635393895, [0.0, 19031.914356577647, 6.373971587192394e6], 6.374000000635394e6) GeoPoint(89.82891902493307, 90.0, 28000.00063291099, [0.0, 19106.914019079923, 6.398971474693153e6], 6.399000000632911e6)], allGridsInCartesian = localCoord2D[localCoord2D(Integer[1, 1], [-90.0, -1.2772e7], [-0.9999999999011541, 1.4060303076323332e-5], [-1.4060303076323332e-5, -0.9999999999011541]) localCoord2D(Integer[1, 2], [-90.0, -1.2747e7], [-0.9999999999003775, 1.4115432871939063e-5], [-1.411543287193906e-5, -0.9999999999003772]) … localCoord2D(Integer[1, 512], [-90.0, 3000.0], [0.9999999999003149, 1.4119861937790054e-5], [-1.4119861937790054e-5, 0.9999999999003149]) localCoord2D(I

In [8]:
# getting model parameters on the 2D disk
seismicModel=lazyProduceOrLoad("seismicModel2DGlobal",getParamsAndTopo,boxGrids.allGridsInGeoPoints,boxGrids.effectiveRadii,2.0)

Computing seismicModel2DGlobal


┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/topoViaGMT/topoViaGMT_43c0ac82.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:106


"not yet coded for your planet" = "not yet coded for your planet"


grdblend [NOTICE]: Remote data courtesy of GMT data server oceania [http://oceania.generic-mapping-tools.org]

grdblend [NOTICE]: SRTM15 Earth Relief v2.7 at 01x01 arc minutes reduced by Gaussian Cartesian filtering (5.2 km fullwidth) [Tozer et al., 2019].
grdblend [NOTICE]:   -> Download 30x30 degree grid tile (earth_relief_01m_g): S90W090
┌ Warning: The Git repository ('/Users/nobuaki/Documents/Github/flexOPT') is dirty! Appending -dirty to the commit ID.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_tools.jl:71
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/topoViaGMT/topoViaGMT_43c0ac82.jld2 saved.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:115


(ρ = [0.0 0.0 … 0.8063660086637596 0.0], Vpv = [0.0 0.0 … 1.7988164808653098 0.0], Vph = [0.0 0.0 … 1.7988164808653098 0.0], Vsv = [0.0 0.0 … 0.9924504722015502 0.0], Vsh = [0.0 0.0 … 0.9924504722015502 0.0], Qμ = [0.0 0.0 … 186.08446353779067 0.0], Qκ = [0.0 0.0 … 17933.26989190945 0.0], QμPower = [0.0 0.0 … 0.0 0.0], QκPower = [0.0 0.0 … 0.0 0.0], η = [0.0 0.0 … 0.31014077256298445 0.0])

# Reading models from Takashi and Paul

Paul is ready to give us more models whenever we want but the data are confidential and I cannot share on the Github